In [1]:
## LIBRARIES
import pandas as pd
import numpy as np
import pdfplumber
import os
import sys

In [9]:
## DEFINE CONSTANTS & PARAMETERS

# Define the project root directory
root_dir = '/Users/giorgiobolchi2/Documents/GitHub/jrc-egd/LLM/'

# File paths
report1_dir = '/Users/giorgiobolchi2/Documents/GitHub/jrc-egd/LLM/Data/REPORT_1/report1.pdf'
report1_annex_dir = '/Users/giorgiobolchi2/Documents/GitHub/jrc-egd/LLM/Data/REPORT_1/report1_annex.pdf'
target_data_dir = '/Users/giorgiobolchi2/Documents/GitHub/jrc-egd/LLM/Data/targets_data_250.csv'

# LLM parameters
seed = None 
temperature = 0.2 
model = "llama-3.3-70b-instruct"  # other models are listed in Documentation/JRC_chat_models.csv
date= '0210' # to indicate date in filenames
output_dir = f'/Users/giorgiobolchi2/Documents/GitHub/jrc-egd/LLM/Data/Outputs/{date}/'
# prompt = see below


In [12]:
## IMPORT DATA & FUNCTIONS


sys.path.insert(0, root_dir) # define absolute path

# Load API and import request function
from Code.API import get_chat_response 

# Import all target data (target_code + target_content)
target_data = pd.read_csv(target_data_dir, sep=";") # note: the separator is a semicolon (;) and not a comma (,)

# Load NACE data
from Data.NACEdata import NACElevel1, NACElevel2, NACElevel3

# Import report1 and report1_annexes as pdf and convert to plain text
with pdfplumber.open(report1_dir) as pdf:
    report1 = ""
    for page in pdf.pages:
        report1 += page.extract_text()

with pdfplumber.open(report1_annex_dir) as pdf:
    report1_annex = ""
    for page in pdf.pages:
        report1_annex += page.extract_text()

# Clean the text
report1 = report1.strip()  # Remove leading and trailing whitespaces
report1 = report1.replace("\n", " ") # Replace new lines with spaces
report1 = report1.replace("\t", " ") # Replace tabs with spaces

report1_annex = report1_annex.strip()
report1_annex = report1_annex.replace("\n", " ")
report1_annex = report1_annex.replace("\t", " ")

print("Report 1:", report1)

Report 1: ISSN 1831-9424 DELIVERING THE EU GREEN DEAL Progress towards targets 2025 EUR 40167This document is a publication by the Joint Research Centre (JRC), the European Commission’s science and knowledge service. It aims to provide evidence-based scientific support to the European policymaking process. The contents of this publication do not necessarily reflect the position or opinion of the European Commission. Neither the European Commission nor any person acting on behalf of the Commission is responsible for the use that might be made of this publication. For information on the methodology and quality underlying the data used in this publication for which the source is neither Eurostat nor other Commission services, users should contact the referenced source. The designations employed and the presentation of material on the maps do not imply the expression of any opinion whatsoever on the part of the European Union concerning the legal status of any country, territory, city or a

In [ ]:
# GENERATE ANSWERS


# Select data to loop through
TA = (['TA1','TA1','TA1',   # n = 3xTA = triplicats per TA
       'TA2','TA2','TA2',
       'TA3','TA3','TA3',
       'TA4','TA4','TA4',
       'TA5','TA5','TA5',
       'TA6','TA6','TA6',
       'TA7','TA7','TA7',]) 
#TA = ['TA1','TA1','TA1'] # if want to test on only TA (smaller subset to go faster)


# Model parameters
seed = seed 
temperature = temperature
model = model # as defined above
date= date # to indicate date in filenames
output_dir = output_dir


# (Loop tools)
loop_counter = 0
answers_content = []
answers_metadata = pd.DataFrame(columns=["TA",          # create empty panda dataframe with the following columns so to gather a bit more data on the responses and ultimately try to assess consistency
                                         "replicate",
                                         "seed",
                                         "temperature",
                                         "system_fingerprint", 
                                         "prompt_tokens", 
                                         "completion_tokens"])  

# Loop
for x in range(len(TA)):

    # Subset data to avoid overloading the GPT
    target_subset = target_data[target_data['target_code'].str.contains(TA[x])]  # subset rows containing one of the characters in  TA[] (ie, select only a specific TA and its targets, because selecting everything in one go is too big for the AI to process)                                                                                                                                          # eg: TA[0] = 'TA1'
    target_list = [f"{row['target_code']}: {row['target_content']}" for index, row in target_subset.iterrows()] # Concatenate target_code and target_content into a list so that it can be added to the prompt as text
    
    
    # Define request
    prompt = f'''Hello,

            Data input: please get acquainted with the following data:
            - NACE classification categories:  {NACElevel1} + {NACElevel2} + {NACElevel3}.
            - List of targets: {target_list}.
            - Report n°1 about "DELIVERING THE EU GREEN DEAL Progress towards targets (2025)": {report1} + {report1_annex}
            
            Task: 
            - In the context of report n°1, for each target, analyse its content description and assign to each target a NACE category for each level (1,2,3). 

            Answer format: provide your answer as a table in csv format please (separator: ";"), with the following columns:
            - target_code (e.g., TA1.9)
            - target_content (e.g., The contribution of the sectors covered by the EU ETS with respect to the EU Climate ambition should be of -62 % compared to 2005 (increasing the linear emissions reduction factor from 2.2 % per year up to 4.4 %)) 
            - NACE_level1 (e.g., D - Electricity, Gas, Steam and Air Conditioning Supply)
            - NACE_level1_extra1 (e.g.if other categories overlap)
            - NACE_level1_extra2 (e.g.if other categories overlap)
            - NACE_level2 (e.g., D35 - Electricity, gas, steam and air conditioning supply)
            - NACE_level2_extra1 (e.g.if other categories overlap)
            - NACE_level2_extra2 (e.g.if other categories overlap)
            - NACE_level_3 (e.g., D35.1 - Electric power generation, transmission and distribution)
            - NACE_level_3_extra1 (e.g.if other categories overlap)
            - NACE level3_extra2 (e.g.if other categories overlap)
            - justification
            - confidence_score (e.g. confidence value from 0 to 10 about the assignation choices that are made)

            Specifications:
            - If there is some overlap, add the multiple possible fitting NACE categories (up to maximum 3 per NACE level). 
            - Include the name of the NACE categories.
            - Don't forget to provide the title of the target. 
            - If there is no target content, do not invent new content, just state it as empty.
            - For each target, write one to two sentences justifying your choice.
            - Output only the csv table and no additional commentary text.

            Thank you.'''

    # Generate answer
    answer= get_chat_response(prompt=prompt,
                              seed=seed,
                              model=model,
                              temperature=temperature  # The temperature parameter influences the randomness of the generated responses. A higher value, such as 0.8, makes the answers more diverse, while a lower value, like 0.2, makes them more focused and deterministic.
                              )

    answers_content.append((f'{TA[x]}.{loop_counter+1}', answer['response_content'])) # add the different replicats for answers over a single TA in a same list so i can analyse the similarity later
    answers_metadata.loc[x] = (f'{TA[x]}', #TA code
                               f'{loop_counter+1}', # replicate nbr
                               seed, 
                               temperature, 
                               answer["system_fingerprint"],
                               answer["prompt_tokens"],
                               answer["completion_tokens"]
                               ) 
    
    # Save response as csv file
    output_name = f'{date}output_{TA[x]}.{loop_counter+1}_s{seed}_t{temperature}.csv'

    with open((os.path.join(output_dir, output_name)), 'w') as f:
         f.write(answer["response_content"])

    # (Extra loop tools)
    loop_counter += 1         # incremental loop counter that resets to 0 every 3 loops so that it can add the ".1,2,3" at the end of each triplicats file names
    if loop_counter % 3 == 0:
        loop_counter = 0


# Save triplicats metadata as csv
answers_metadata.to_csv(path_or_buf= f'/Users/giorgiobolchi2/Documents/JRC/LLM/Data/Outputs/{date}/{date}output_s{seed}_t{temperature}_metadata.csv', 
                        sep=';', 
                        index=False)


In [ ]:
# CLEAN CSV FILES

# This code block cleans the generated csv files from their headers and tails, which are not needed for the analysis.

output_dir = output_dir

for filename in os.listdir(output_dir):

    if 'TA' in filename and '.csv' in filename:     # Check if the file name contains the characters 'TA'
        
        # Open the file for reading and writing
        with open(f'{output_dir}{filename}', 'r+') as file:  # r+ = mode to open the file for both reading and writing
            lines = file.readlines()
            lines = lines[1:-1] # remove first and last row, which are usually just '```' characters.
            file.seek(0)  # move the file pointer to the beginning
            file.writelines(lines) # write the modified lines back to the file.
            file.truncate() # set the length of the file to the current position of the file pointer.


In [ ]:
# TRIPLICATS COMPARISON

# This code block compares the consistency of answers generated by the language model across three replicates for each target (TA), 
# identifying similarities and differences in the assigned NACE categories. The output shows the answers consistency across triplicats, and the distributions of missing values.
# I reccomend to read the comparison dataframe through the DataWrangler tool in VScode, as it autoamtically provides some summarized statistics on top of the dataframe.


# Important note: you need to clean the generated csv files manually before running this following code block: 
#                 - if previous cleaning loop didn't work (it should work) -> remove the first line containing the characters ```csv and the last line with ```
#                 - it can happen that some columns (especially justifications and confidence_scores) are moved one cell to the right or left, move them back in the right place


date= date
seed= seed
temperature = temperature
output_dir = output_dir


#TA = ['TA1','TA2','TA3','TA4','TA5','TA6','TA7']
TA = ['TA5']

for x in range(len(TA)):

    # Define temporary dataframes for respective TA replicates 
    df1 = pd.read_csv(f'{output_directory}{date}output_{TA[x]}.1_s{seed}_t{temperature}.csv', sep=";")
    df2 = pd.read_csv(f'{output_directory}{date}output_{TA[x]}.2_s{seed}_t{temperature}.csv', sep=";")
    df3 = pd.read_csv(f'{output_directory}{date}output_{TA[x]}.3_s{seed}_t{temperature}.csv', sep=";")


    # Create a list of these DataFrames
    dfs = [df1, df2, df3]

    # Create a list to store the comparison tables
    comparisons = []

    # Compare replicats dataframes:
    for i in range(len(dfs)):
        for j in range(i+1, len(dfs)):
            comparison = dfs[i].reset_index(drop=True).eq(dfs[j].reset_index(drop=True)) # Reset the index before comparing (make it easier to compare)
            comparisons.append(comparison) # Add the comparison table to the list

    # Concatenate the comparison tables into a single DataFrame
    df_comparisons = pd.concat(comparisons, ignore_index=True)

    # Get information on the distribution of NA values per columns per replicat
    df_na = pd.DataFrame({f'{TA[x]}.1': df1.isna().sum(), 
                          f'{TA[x]}.2': df2.isna().sum(), 
                          f'{TA[x]}.3': df3.isna().sum()})
    
# Save results as csv
    output_directory = output_directory
    # Comparisons
    df_comparisons.to_csv(f'{output_directory}{date}comparisons_{TA[x]}.csv', sep=';', index=False)
    # NA distributions
    df_na.to_csv(f'{output_directory}{date}NAdistribution_{TA[x]}.csv', sep=';', index=True)